In [1]:
# Cell 1: imports + config
import os, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from statsmodels.tsa.seasonal import STL
from scipy.signal import find_peaks

warnings.filterwarnings("ignore", category=FutureWarning)

# OUTPUT PATHS
RESULTS_DIR = "../results/reviewer_tables"
FIG_DIR     = "../results/reviewer_figures"
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(FIG_DIR, exist_ok=True)

# ---- TIME WINDOW (updated) ----
START_DATE = "2012-01-01 00:00:00"
END_DATE   = '2018-12-31 00:00:00'   # <-- updated as you requested
START_DAY, END_DAY = 0, 2531         # keep your row slice
SPLIT_VLINE = "2018-01-01"           # dashed line for plots
STL_PERIOD  = 365                    # daily seasonality (~annual)

# plotting style (your house style)
plt.rcParams.update({
    "font.size": 14, "axes.labelsize": 16, "axes.titlesize": 18,
    "legend.fontsize": 12, "xtick.labelsize": 12, "ytick.labelsize": 12,
    "lines.linewidth": 2.2
})
PALETTE = {
    "flood_dark": "#d64e3b", "flood_light": "#f3b3b3",
    "rain_dark":  "#2d6faa", "rain_light":  "#b7d1f1",
    "ft_dark":    "#8a1c1c", "ft_light":    "#e7b1b1",
    "rt_dark":    "#0e2451", "rt_light":    "#9eb3d6",
    "frt_dark":   "#102a43", "frt_light":   "#9bb3c8", "grey":"%s" % "#444444"
}


In [7]:
# Cell 2: helpers

def _to_series(df, date_col, value_col):
    s = pd.Series(pd.to_numeric(df[value_col], errors="coerce").values,
                  index=pd.to_datetime(df[date_col]))
    s = s.sort_index().asfreq("D")
    return s.interpolate(limit_direction="both")

def mad(x, scale_to_sd=False):
    med = np.median(x)
    m = np.median(np.abs(x - med))
    return 1.4826*m if scale_to_sd else m

def lag1_autocorr(x):
    return pd.Series(x).astype(float).autocorr(lag=1)

def dispersion_index(x):
    x = np.asarray(x, float)
    mu = np.mean(x)
    return np.var(x, ddof=1)/mu if mu != 0 else np.nan

def stl_features(series, period=STL_PERIOD, robust=True):
    stl = STL(series, period=period, robust=robust)
    res = stl.fit()
    seas, resid = res.seasonal, res.resid
    feats = {
        "mean": float(series.mean()),
        "sd": float(series.std(ddof=1)),
        "iqr": float(np.subtract(*np.percentile(series, [75, 25]))),
        "mad": float(mad(series)),
        "mad_scaled_as_sd": float(mad(series, True)),
        "pct_zeros": float((np.asarray(series)==0).mean()*100),
        "cv": float(series.std(ddof=1)/series.mean()) if series.mean()!=0 else np.nan,
        "dispersion_index": float(dispersion_index(series)),
        "lag1_autocorr": float(lag1_autocorr(series)),
        "stl_amp_peak_trough": float((seas.max()-seas.min())/2.0),
        "stl_amp_iqr": float((np.percentile(seas,75)-np.percentile(seas,25))/2.0),
        "stl_seasonal_sd": float(seas.std(ddof=1)),
        "stl_resid_sd": float(resid.std(ddof=1)),
        "stl_resid_mad": float(mad(resid)),
        "stl_resid_mad_scaled_as_sd": float(mad(resid, True)),
    }
    return res, feats

# Dominant-only version: exactly one peak (the max seasonal point) per year
def yearwise_oscillation_summary(seasonal_component, dominant_only=True):
    """
    If dominant_only=True, returns exactly ONE peak per year (the within-year seasonal maximum).
    If dominant_only=False, counts all local peaks using relaxed rules.
    """
    import numpy as np
    import pandas as pd
    from scipy.signal import find_peaks

    df = seasonal_component.to_frame("seasonal")
    df["year"] = df.index.year
    out = []
    for y, g in df.groupby("year"):
        vals = g["seasonal"].values
        amp  = (np.nanmax(vals) - np.nanmin(vals)) / 2.0

        if dominant_only:
            # exactly one: the date where the seasonal component is max in that year
            peak_ts = g.iloc[vals.argmax()].name
            n_peaks = 1
        else:
            # relaxed detection (can be >1 if shoulders exist)
            prom = np.nanstd(vals) * 0.5 if np.nanstd(vals) > 0 else None
            idx, _ = find_peaks(vals, prominence=prom, distance=120, width=10)
            n_peaks = int(len(idx))
            peak_ts = g.index[idx[np.argmax(vals[idx])]] if len(idx) else g.iloc[vals.argmax()].name

        out.append({
            "year": int(y),
            "n_peaks": int(n_peaks),
            "peak_month": int(peak_ts.month),
            "peak_date": peak_ts,
            "year_amp_peak_trough": float(amp),
            "seasonal_sd_year": float(np.nanstd(vals, ddof=1)),
        })
    return pd.DataFrame(out).sort_values("year")

def compute_error_metrics(y_true, y_pred):
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    mse = np.mean((y_true-y_pred)**2)
    mae = np.mean(np.abs(y_true-y_pred))
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true-y_pred)/np.maximum(y_true,1e-12)))*100
    smape = np.mean(2*np.abs(y_true-y_pred)/(np.abs(y_true)+np.abs(y_pred)+1e-12))*100
    nrmse = rmse/(np.max(y_true)-np.min(y_true)+1e-12)
    rel = np.mean(np.abs((y_true-y_pred)/np.maximum(y_true,1e-12)))*100
    return {"MSE":mse,"MAE":mae,"MAPE (%)":mape,"SMAPE (%)":smape,"RMSE":rmse,"NRMSE":nrmse,"REL (%)":rel}

def normalize_errors(errs, sd_remainder, mad_remainder):
    return {
        **errs,
        "RMSE/SD_remainder": errs["RMSE"]/(sd_remainder if sd_remainder>0 else np.nan),
        "MAE/MAD_remainder": errs["MAE"]/(mad_remainder if mad_remainder>0 else np.nan),
    }

def smart_pick(colnames, prefixes):
    cl = [c.lower() for c in colnames]
    for p in prefixes:
        if p in cl: return colnames[cl.index(p)]
    for p in prefixes:
        for i,c in enumerate(cl):
            if c.startswith(p): return colnames[i]
    return colnames[-1]


In [4]:
# Cell 3: time axis + split
cov_all = pd.read_excel("../data/covariates_date.xlsx")
cov = cov_all[(cov_all["Date"]>=START_DATE) & (cov_all["Date"]<=END_DATE)].loc[START_DAY:END_DAY]
time_axis = pd.to_datetime(cov["Date"].values)

split_dt = pd.to_datetime(SPLIT_VLINE)
train_size = int(np.argmin(np.abs(time_axis - split_dt)))



In [9]:
# --- existing lines (keep) ---
stl_res_gt, feats_gt = stl_features(Ih_true)
gt_feats_df = pd.DataFrame([feats_gt])
#gt_yw = yearwise_oscillation_summary(stl_res_gt.seasonal)
# After you've computed: stl_res_gt, feats_gt, Ih_true, etc.
gt_yw_dom = yearwise_oscillation_summary(stl_res_gt.seasonal, dominant_only=True)
print("Dominant peaks (should be 7):", len(gt_yw_dom))
gt_yw_dom


# --- NEW: robust peak counting rules ---
seasonal = stl_res_gt.seasonal
sd_seasonal = feats_gt["stl_seasonal_sd"]

# 1) strict global-peak detection
#    - require large prominence (>= 0.8*SD of seasonal)
#    - require minimum distance between peaks (~200 days) so we don't count shoulders
#    - require minimum width so spikes don't count
strict_prom = 0.8 * sd_seasonal if sd_seasonal > 0 else None
peaks_idx_strict, _ = find_peaks(
    seasonal.values,
    prominence=strict_prom,
    distance=200,     # ~ > 6 months separation
    width=15          # ~ > 2 weeks wide
)

# 2) dominant peak per year (exactly one per year)
annual_max_idx = seasonal.groupby(seasonal.index.year).idxmax()
annual_max_idx = annual_max_idx.sort_values()
n_years = annual_max_idx.shape[0]

print(f"Strict global peaks: {len(peaks_idx_strict)} (with prominence >= 0.8*SD, distance >= 200d)")
print(f"Dominant (max) seasonal peak per year: {n_years} (expected ≈ number of years)")
print("Years covered:", seasonal.index.min().year, "→", seasonal.index.max().year)

# --- Save CSVs (unchanged) ---
gt_feats_df.to_csv(f"{RESULTS_DIR}/groundtruth_Ih_descriptives_and_STL.csv", index=False)
#gt_yw.to_csv(f"{RESULTS_DIR}/groundtruth_Ih_yearwise_oscillation.csv", index=False)
gt_yw_dom.to_csv(f"{RESULTS_DIR}/groundtruth_Ih_yearwise_oscillation_DOMINANT.csv", index=False)


# --- NEW: save peak dates for transparency (reviewer-friendly) ---
peaks_df = pd.DataFrame({
    "peak_type": ["strict"] * len(peaks_idx_strict) + ["annual_max"] * len(annual_max_idx),
    "date": list(seasonal.index[peaks_idx_strict]) + list(annual_max_idx.values),
    "value": list(seasonal.values[peaks_idx_strict]) + list(seasonal.loc[annual_max_idx.values].values),
})
peaks_df = peaks_df.sort_values("date").reset_index(drop=True)
peaks_df.to_csv(f"{RESULTS_DIR}/groundtruth_Ih_seasonal_peaks.csv", index=False)

# quick display
gt_feats_df, gt_yw_dom.head(), peaks_df.head()


Dominant peaks (should be 7): 7
Strict global peaks: 7 (with prominence >= 0.8*SD, distance >= 200d)
Dominant (max) seasonal peak per year: 7 (expected ≈ number of years)
Years covered: 2012 → 2018


(        mean         sd        iqr       mad  mad_scaled_as_sd  pct_zeros  \
 0  22.969698  14.926233  15.025098  6.584166          9.761685        0.0   
 
          cv  dispersion_index  lag1_autocorr  stl_amp_peak_trough  \
 0  0.649823          9.699406       0.999393            21.156017   
 
    stl_amp_iqr  stl_seasonal_sd  stl_resid_sd  stl_resid_mad  \
 0     7.064224         9.559894       9.39282       1.028971   
 
    stl_resid_mad_scaled_as_sd  
 0                    1.525553  ,
    year  n_peaks  peak_month  peak_date  year_amp_peak_trough  \
 0  2012        1          10 2012-10-19             21.156017   
 1  2013        1          10 2013-10-19             19.256440   
 2  2014        1          10 2014-10-18             17.600398   
 3  2015        1          11 2015-11-03             16.410983   
 4  2016        1          10 2016-10-12             16.198489   
 
    seasonal_sd_year  
 0         11.446866  
 1         10.545633  
 2          9.868307  
 3         

In [10]:
# Cell X: Compare seasonal-peak values vs raw Ih (and components)

# seasonal, trend, remainder from the earlier STL fit
seasonal = stl_res_gt.seasonal
trend    = stl_res_gt.trend
remainder= stl_res_gt.resid

# Use the dominant-per-year peak dates you already computed
gt_yw_dom = yearwise_oscillation_summary(stl_res_gt.seasonal, dominant_only=True)

rows = []
for dt in pd.to_datetime(gt_yw_dom["peak_date"].values):
    s_val = float(seasonal.loc[dt])
    tr_val = float(trend.loc[dt])
    r_val = float(remainder.loc[dt])
    ih_val = float(Ih_true.loc[dt])  # raw Ih at that date
    recon = tr_val + s_val + r_val   # should be ~ ih_val
    rows.append({
        "peak_date": dt,
        "seasonal_peak": s_val,
        "trend": tr_val,
        "remainder": r_val,
        "Ih_at_date": ih_val,
        "trend+seasonal+remainder": recon,
        "reconstruction_error": ih_val - recon
    })

Ih_at_seasonal_peaks = pd.DataFrame(rows).sort_values("peak_date")
display(Ih_at_seasonal_peaks)

# Optional: also get the dominant raw-Ih peak per year and compare dates
raw_peak_rows = []
gt_df = Ih_true.to_frame("Ih"); gt_df["year"] = gt_df.index.year
for y, g in gt_df.groupby("year"):
    idx = g["Ih"].idxmax()
    raw_peak_rows.append({"year": int(y), "raw_Ih_peak_date": idx, "raw_Ih_peak_value": float(g.loc[idx, "Ih"])})
raw_Ih_peaks = pd.DataFrame(raw_peak_rows)

# Join the two to see how close seasonal-peak dates are to raw Ih peak dates
cmp = gt_yw_dom.merge(raw_Ih_peaks, on="year", how="left")
cmp["days_diff"] = (pd.to_datetime(cmp["raw_Ih_peak_date"]) - pd.to_datetime(cmp["peak_date"])).dt.days
display(cmp[["year","peak_date","raw_Ih_peak_date","days_diff","year_amp_peak_trough"]])

# Save for the manuscript
Ih_at_seasonal_peaks.to_csv(f"{RESULTS_DIR}/groundtruth_Ih_at_seasonal_peak_dates.csv", index=False)
cmp.to_csv(f"{RESULTS_DIR}/groundtruth_Ih_dominant_seasonal_vs_raw_peaks.csv", index=False)


peak_date  seasonal_peak      trend  remainder  Ih_at_date  \
0 2012-10-19      28.939262  27.424482   0.269941   56.633685   
1 2013-10-19      26.099213  19.703313  21.225299   67.027825   
2 2014-10-18      23.508194  16.815098 -17.892365   22.430927   
3 2015-11-03      20.913592  19.021920  -6.400805   33.534707   
4 2016-10-12      20.227711  23.290973   2.969396   46.488080   
5 2017-10-11      19.376769  23.488711  37.094804   79.960284   
6 2018-10-11      18.629015  20.246572  -0.445874   38.429712   

   trend+seasonal+remainder  reconstruction_error  
0                 56.633685          0.000000e+00  
1                 67.027825          0.000000e+00  
2                 22.430927          0.000000e+00  
3                 33.534707          7.105427e-15  
4                 46.488080         -7.105427e-15  
5                 79.960284          0.000000e+00  
6                 38.429712          0.000000e+00

year  peak_date raw_Ih_peak_date  days_diff  year_amp_peak_trough
0  2012 2012-10-19       2012-10-19          0             21.156017
1  2013 2013-10-19       2013-10-17         -2             19.256440
2  2014 2014-10-18       2014-11-27         40             17.600398
3  2015 2015-11-03       2015-10-15        -19             16.410983
4  2016 2016-10-12       2016-10-14          2             16.198489
5  2017 2017-10-11       2017-10-26         15             15.955572
6  2018 2018-10-11       2018-10-11          0             15.725197

In [14]:
sequence_length = 1
train_size = 2191

# Load Ih prediction data (for training part) from Excel files
df_flood_train = pd.read_excel("../results/predictions_plot_1stmodel_flood_seqlen1_oneinitcond_train.xlsx")
df_floodIh_train = pd.read_excel("../results/predictions_plot_1stmodel_floodIh_seqlen1_oneinitcond_train.xlsx")

df_rain_train = pd.read_excel("../results/predictions_plot_1stmodel_rain_seqlen1_oneinitcond_train.xlsx")
df_raincases_train = pd.read_excel("../results/predictions_plot_1stmodel_raincases_seqlen1_oneinitcond_train.xlsx")

df_floodTEMP_train = pd.read_excel("../results/predictions_plot_1stmodel_floodandTEMP_seqlen1_oneinitcond_train.xlsx")
df_floodTEMPcases_train = pd.read_excel("../results/predictions_plot_1stmodel_floodTEMPcases_seqlen1_oneinitcond_train.xlsx")

df_rainTEMP_train = pd.read_excel("../results/predictions_plot_1stmodel_rainTEMP_seqlen1_oneinitcond_train.xlsx")
df_rainTEMPcases_train = pd.read_excel("../results/predictions_plot_1stmodel_rainTEMPcases_seqlen1_oneinitcond_train.xlsx")

df_floodrainTEMP_train = pd.read_excel("../results/predictions_plot_1stmodel_floodrainTEMP_seqlen1_oneinitcond_train.xlsx")
df_floodrainTEMPcases_train = pd.read_excel("../results/predictions_plot_1stmodel_floodrainTEMPcases_seqlen1_oneinitcond_train.xlsx")


# Load Ih prediction data (for testing part) from Excel files
df_flood_test = pd.read_excel("../results/predictions_plot_1stmodel_flood_seqlen1_oneinitcond_test.xlsx")
df_floodIh_test = pd.read_excel("../results/predictions_plot_1stmodel_floodIh_seqlen1_oneinitcond_test.xlsx")
#df_floodIh_test = pd.read_excel("predictions_plot_1stmodel_floodIh_seqlen1_oneinitcond_test_useinitcond.xlsx")

df_rain_test = pd.read_excel("../results/predictions_plot_1stmodel_rain_seqlen1_oneinitcond_test.xlsx")
df_raincases_test = pd.read_excel("../results/predictions_plot_1stmodel_raincases_seqlen1_oneinitcond_test.xlsx")

df_floodTEMP_test = pd.read_excel("../results/predictions_plot_1stmodel_floodandTEMP_seqlen1_oneinitcond_test.xlsx")
#df_floodTEMPcases_train = pd.read_excel("predictions_plot_1stmodel_floodTEMPcases_seqlen1_oneinitcond_train.xlsx")
df_floodTEMPcases_test = pd.read_excel("../results/predictions_plot_1stmodel_floodTEMPcases_seqlen1_oneinitcond_test.xlsx")
#df_floodTEMPcases_test = pd.read_excel("predictions_plot_1stmodel_floodTEMPcases_seqlen1_oneinitcond_test_useinitcond.xlsx")

df_rainTEMP_test = pd.read_excel("../results/predictions_plot_1stmodel_rainTEMP_seqlen1_oneinitcond_test.xlsx")
df_rainTEMPcases_test = pd.read_excel("../results/predictions_plot_1stmodel_rainTEMPcases_seqlen1_oneinitcond_test.xlsx")

df_floodrainTEMP_test = pd.read_excel("../results/predictions_plot_1stmodel_floodrainTEMP_seqlen1_oneinitcond_test.xlsx")
df_floodrainTEMPcases_test = pd.read_excel("../results/predictions_plot_1stmodel_floodrainTEMPcases_seqlen1_oneinitcond_test.xlsx")

In [23]:
# Cell 5 testing metrics from arrays + normalized by STL remainder (test slice)

# 1) define the testing window using your indices
test_start = sequence_length + train_size

# ground-truth for test period (use the same source as your models use)
# prefer Ih_true (aligned daily series) if available
y_groundtruth = Ih_true.values[test_start:]
# if you strictly want to use the raw dataframe `data`, uncomment the next line:
# y_groundtruth = data["Ih"].values[test_start:]

# 2) build scenario dict from the *test* DataFrames you already loaded
scenarios = {
    "Flood":                       df_flood_test["Ih_pred"].values,
    "Flood + Ih":                  df_floodIh_test["Ih_pred"].values,
    "Rain":                        df_rain_test["Ih_pred"].values,
    "Rain + Ih":                   df_raincases_test["Ih_pred"].values,
    "Flood + Temp":                df_floodTEMP_test["Ih_pred"].values,
    "Flood + Temp + Ih":           df_floodTEMPcases_test["Ih_pred"].values,
    "Rain + Temp":                 df_rainTEMP_test["Ih_pred"].values,
    "Rain + Temp + Ih":            df_rainTEMPcases_test["Ih_pred"].values,
    "Flood + Rain + Temp":         df_floodrainTEMP_test["Ih_pred"].values,
    "Flood + Rain + Temp + Ih":    df_floodrainTEMPcases_test["Ih_pred"].values,
}

#Aj Ih
df_ode_results_previous_work = pd.read_excel('../data/data.xlsx')
data_previous_work = df_ode_results_previous_work[(df_ode_results_previous_work['Date'] >= START_DATE) & (df_ode_results_previous_work['Date'] <= END_DATE)]

data_previous_work = data_previous_work.loc[START_DAY:END_DAY]
#data_previous_work.columns
# Optional: add Previous work if available
try:
    scenarios["Previous work"] = data_previous_work["Ih"].values[test_start:]
except Exception:
    pass  # it's fine if not present

# helper: align lengths safely (in case any vector is off by a few days)
def _align_len(y_true, y_pred):
    n = int(min(len(y_true), len(y_pred)))
    return np.asarray(y_true[:n], float), np.asarray(y_pred[:n], float)


In [24]:
# --- Use WHOLE-SERIES normalization from Cell 4 (previous values) ---
sd_rem_norm  = float(feats_gt["stl_resid_sd"])         
mad_rem_norm = float(feats_gt["stl_resid_mad"])       
print(f"Normalizing with WHOLE remainder: SD={sd_rem_norm:.3f}, MAD={mad_rem_norm:.3f}")


Normalizing with WHOLE remainder: SD=9.393, MAD=1.029


In [25]:
rows = []
for label, y_pred_full in scenarios.items():
    y_t, y_p = _align_len(y_groundtruth, y_pred_full)   # or your direct arrays
    errs  = compute_error_metrics(y_t, y_p)
    nerrs = normalize_errors(errs, sd_rem_norm, mad_rem_norm)
    rows.append({"model": label, **nerrs})

metrics_df = pd.DataFrame(rows).set_index("model").sort_index()
display(metrics_df)

metrics_df.to_csv(f"{RESULTS_DIR}/errors_test_WHOLEnorm.csv")
errors_table = metrics_df.copy()
errors_table.to_csv(f"{RESULTS_DIR}/errors_normalized_by_STLremainder_WHOLE.csv")


MSE        MAE   MAPE (%)  SMAPE (%)  \
model                                                                   
Flood                      92.081960   6.344180  30.362669  24.668992   
Flood + Ih                 85.855329   8.584267  52.334156  38.836711   
Flood + Rain + Temp       269.992018  12.241851  60.657770  42.038656   
Flood + Rain + Temp + Ih  183.052091  13.114790  75.436609  52.281082   
Flood + Temp              116.357866   7.899420  40.542482  31.288369   
Flood + Temp + Ih          35.411264   5.317658  33.837158  26.876433   
Previous work              91.785423   6.318918  30.467564  24.688475   
Rain                       61.063753   6.085517  29.130479  24.718549   
Rain + Ih                  47.160448   6.199332  39.132213  30.387059   
Rain + Temp               325.045871  14.452991  83.341660  52.431796   
Rain + Temp + Ih          130.721868  10.936036  64.203003  46.069058   

                               RMSE     NRMSE    REL (%)  RMSE/SD_remainder  \
model                                                                         
Flood                      9.595935  0.322850  30.362669           1.021624   
Flood + Ih                 9.265815  0.311743  52.334156           0.986479   
Flood + Rain + Temp       16.431434  0.552826  60.657770           1.749361   
Flood + Rain + Temp + Ih  13.529674  0.455198  75.436609           1.440427   
Flood + Temp              10.786930  0.362920  40.542482           1.148423   
Flood + Temp + Ih          5.950736  0.200209  33.837158           0.633541   
Previous work              9.580471  0.322329  30.467564           1.019978   
Rain                       7.814330  0.262909  29.130479           0.831947   
Rain + Ih                  6.867347  0.231048  39.132213           0.731127   
Rain + Temp               18.029029  0.606576  83.341660           1.919448   
Rain + Temp + Ih          11.433366  0.384669  64.203003           1.217245   

                          MAE/MAD_remainder  
model                                        
Flood                              6.165557  
Flood + Ih                         8.342573  
Flood + Rain + Temp               11.897176  
Flood + Rain + Temp + Ih          12.745537  
Flood + Temp                       7.677008  
Flood + Temp + Ih                  5.167937  
Previous work                      6.141006  
Rain                               5.914176  
Rain + Ih                          6.024787  
Rain + Temp                       14.046060  
Rain + Temp + Ih                  10.628126

In [26]:
ranked = errors_table.sort_values("RMSE/SD_remainder")[["RMSE/SD_remainder","MAE/MAD_remainder","RMSE","MAE","NRMSE"]]
display(ranked)
ranked.to_csv(f"{RESULTS_DIR}/errors_test_ranked_by_RMSE_over_SDwhole.csv")


RMSE/SD_remainder  MAE/MAD_remainder       RMSE  \
model                                                                       
Flood + Temp + Ih                  0.633541           5.167937   5.950736   
Rain + Ih                          0.731127           6.024787   6.867347   
Rain                               0.831947           5.914176   7.814330   
Flood + Ih                         0.986479           8.342573   9.265815   
Previous work                      1.019978           6.141006   9.580471   
Flood                              1.021624           6.165557   9.595935   
Flood + Temp                       1.148423           7.677008  10.786930   
Rain + Temp + Ih                   1.217245          10.628126  11.433366   
Flood + Rain + Temp + Ih           1.440427          12.745537  13.529674   
Flood + Rain + Temp                1.749361          11.897176  16.431434   
Rain + Temp                        1.919448          14.046060  18.029029   

                                MAE     NRMSE  
model                                          
Flood + Temp + Ih          5.317658  0.200209  
Rain + Ih                  6.199332  0.231048  
Rain                       6.085517  0.262909  
Flood + Ih                 8.584267  0.311743  
Previous work              6.318918  0.322329  
Flood                      6.344180  0.322850  
Flood + Temp               7.899420  0.362920  
Rain + Temp + Ih          10.936036  0.384669  
Flood + Rain + Temp + Ih  13.114790  0.455198  
Flood + Rain + Temp       12.241851  0.552826  
Rain + Temp               14.452991  0.606576